In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import smtplib
import os

from sklearn.cluster import KMeans
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from keys import keys

sns.set_theme(style="darkgrid")

In [9]:
toolkit = pd.read_csv("CikguHub_Marketing Planner_2023 - PBD Toolkit Responses.csv")
registraton = pd.read_csv("CikguHub_Marketing Planner_2023 - Registration Form.csv")

In [10]:
# slim down the column names so its readable
tool_users = pd.DataFrame({})
tool_users['teaching_level'] = toolkit[['1. Anda mengajar di sekolah jenis? / Which school are you teaching in?']]
tool_users['experience'] = toolkit[['2. Berapakah tahun anda menjadi pendidik? / How long have you been an educator?']]
tool_users['role'] = toolkit[['3. Apakah jawatan anda di sekolah? / What is your role in school?']]
tool_users['wanted_skills'] = toolkit[['4. Apakah kemahiran yang anda ingin bangunkan? / What are the skills you wish to develop?']]
reg_users = pd.DataFrame({})
reg_users['teaching_level'] = registraton[['1. Anda mengajar di sekolah jenis? / Which school are you teaching in?']]
reg_users['experience'] = registraton[['2. Berapakah tahun anda menjadi pendidik? / How long have you been an educator?']]
reg_users['role'] = registraton[['3. Apakah jawatan anda di sekolah? / What is your role in school?']]
reg_users['wanted_skills'] = registraton[['4. Apakah kemahiran yang anda ingin bangunkan? / What are the skills you wish to develop?']]

# ok let's encode the responses now so we can read them easier
skills_code = {'KemahiranMengajar/TeachingSkills' : 'T', 
              'Bimbingan&Pementoran/Coaching&Mentoring': 'C',
              'Kepimpinan/Leadership': 'L', 
              'KemahiranDigital/DigitalSkills(contoh:aplikasiMicrosoftWord/Excel/PowerPointdanGoogleDoc/Sheet/Slide)': 'D',
              'KemahiranMultimedia/MultimediaSkills(contoh:pembangunanvideo)': 'M'}

exp_code = {'Kurang daripada 1 tahun / Less than 1 year': 0,
            '1 hingga 5 tahun / 1 to 5 years' : 1,
            '6 hingga 10 tahun / 6 to 10 years': 2,
           'Lebih daripada 10 tahun / More than 10 years': 3}

role_code = {'GuruAkademikBiasa/AcademicTeacher': 'T', 
             'KetuaPanitia/PanelHead': 'P', 
             'Officer': 'O'}

level_code = {'Other:': -1,
              'Saya bukan seorang cikgu / I am not a teacher': 0,
              'Sekolah Kebangsaan / National Primary School': 1,
              'Sekolah Menengah Kebangsaan / National Secondary School': 2}

def code_responses(df):

    coded_exp = []
    coded_skills = []
    coded_roles = []
    coded_level = []

    for i, col in df.iterrows(): # read through every row in df

        # grab column data for the row
        lvl = col['teaching_level']
        exp = col['experience']
        skills = col['wanted_skills'].split(',')
        roles = str(col['role']).split(',')
        
        coded_level.append(level_code[lvl]) # pass through the school level they teach

        if exp in exp_code.keys(): # this if/else handles other/nan input
            coded_exp.append(exp_code[exp])
        else:
            coded_exp.append(exp)

        for j in range(len(skills)): # look at each skill selected
            s = skills[j].replace(' ', '')
            if s in skills_code.keys():
                skills[j] = s.replace(s, skills_code[s])
            else:
                skills[j] = '*' # for now mark free response with star
        coded_skills.append(skills)

        for j in range(len(roles)): # look at each role selected
            r = roles[j].replace(' ', '')
            if r in role_code.keys():
                roles[j] = r.replace(r, role_code[r])
            else:
                roles[j] = '*' # for now mark free response with star
        coded_roles.append(roles)
        
    return coded_exp, coded_skills, coded_roles, coded_level


In [11]:

coded_exp1, coded_skills1, coded_roles1, coded_level1 = code_responses(tool_users)
coded_exp2, coded_skills2, coded_roles2, coded_level2 = code_responses(reg_users)

In [12]:
user_model = pd.DataFrame({})
user_model['experience'] = coded_exp1 + coded_exp2
user_model['wanted_skills'] = coded_skills1 + coded_skills2
user_model['roles'] = coded_roles1 + coded_roles2
user_model['teaching_level'] = coded_level1 + coded_level2

# first step is to one hot encode data since several columns are categorical
one_hot_um = pd.DataFrame({})
one_hot_um['experience'] = user_model['experience']
one_hot_um['teaching_level'] = user_model['teaching_level']
one_hot_um['T'] = [1 if 'T' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['C'] = [1 if 'C' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['L'] = [1 if 'L' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['D'] = [1 if 'D' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['M'] = [1 if 'M' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['*'] = [1 if '*' in i else 0 for i in user_model['wanted_skills']]
one_hot_um['roles_T'] = [1 if 'T' in i else 0 for i in user_model['roles']]
one_hot_um['roles_P'] = [1 if 'P' in i else 0 for i in user_model['roles']]
one_hot_um['roles_O'] = [1 if 'O' in i else 0 for i in user_model['roles']]
one_hot_um['roles_*'] = [1 if '*' in i else 0 for i in user_model['roles']]

# user_interests = LearnerModel.skill_interests
# might make more sense to store user interests as a list or dict than a text field
# but this function will transform text from response form into a list of letters
def code_user_interests(user_interests):

    skills_code = {'KemahiranMengajar/TeachingSkills' : 'T', 
                  'Bimbingan&Pementoran/Coaching&Mentoring': 'C',
                  'Kepimpinan/Leadership': 'L', 
                  'KemahiranDigital/DigitalSkills(contoh:aplikasiMicrosoftWord/Excel/PowerPointdanGoogleDoc/Sheet/Slide)': 'D',
                  'KemahiranMultimedia/MultimediaSkills(contoh:pembangunanvideo)': 'M'}

    skills = user_interests.split(',')
    coded_skills = []
    for j in range(len(skills)): # look at each skill selected
        s = skills[j].replace(' ', '')
        if s in skills_code.keys():
            skills[j] = s.replace(s, skills_code[s])
        else:
            skills[j] = '*' # for now mark free response with star
        coded_skills.append(skills)
    return coded_skills

def create_user_df(max_track_num, ranking, user_open_tracks, user_interests, user_engagement, cluster_engagement):
    
    df = pd.DataFrame({'Open Tracks': user_open_tracks, 'Max': max_track_num, 'Ranking': ranking, 
                       'Cluster': cluster_engagement, 'User Interest': user_interests, 
                       'User Engagement': user_engagement})
    df['Avaliable'] = [1 if row['Open Tracks'] < row['Max'] else 0 for i, row in df.iterrows()]
    
    return df

def recommend_videos(df):
    
    feature_order = ['Open Tracks', 'User Engagement', 'Cluster', 'Ranking'] # order in which we consider features
    
    avaliable_tracks = df[df['Avaliable'] ==1]
    user_chosen_recs = avaliable_tracks[df['User Interest'] ==1]
    sorted_recs = list(user_chosen_recs.sort_values(feature_order, ascending=False).index)
    
    if len(sorted_recs) == 2:
        return sorted_recs
    elif len(user_chosen_recs) > 2:
        return sorted_recs[0:2]
    else:
        n = 2 - len(sorted_recs)
        other_recs = avaliable_tracks[df['User Interest'] == 0]
        other_recs = list(other_recs.sort_values(feature_order, ascending=False).index)
        return sorted_recs + other_recs[0:n]
    
# sample system data
max_track_num = {'T':5, 'C':2, 'L':3, 'D':3, 'M':2} # number of tracks for each interest cat.
ranking = one_hot_um.mean()[2:7] # can replace this ranking with any ranking we want

# sample user data
user_open_tracks = {'T':2, 'C':1, 'L':2, 'D':0, 'M':0}
user_interests = one_hot_um.iloc[1][2:7] # these are input by users and can be changed at any time
user_engagement = {'T':0.4, 'C':0.7, 'L':0.2, 'D':0.6, 'M':0.4} # we need to calculate this metric
cluster_engagement = {'T':0.8, 'C':0.2, 'L':0.4, 'D':0.95, 'M':0.6} # we need to calculate this metric

df = create_user_df(max_track_num, ranking, user_open_tracks, user_interests, user_engagement, cluster_engagement)
df

,Open Tracks,Max,Ranking,Cluster,User Interest,User Engagement,Avaliable
T,2,5,0.846154,0.80,0,0.4,1
C,1,2,0.538462,0.20,0,0.7,1
L,2,3,0.576923,0.40,0,0.2,1
D,0,3,0.807692,0.95,1,0.6,1
M,0,2,0.807692,0.60,0,0.4,1


In [13]:
recommend_videos(df)

['D', 'T']

### Email System

In [20]:
subject = "[Example] CikguHub: Here's what you should watch next!"
# body = "Dear X,\n\nThank you for being one of our first supporters! Part of being a CikguHub member is getting personalized video recommendations based on your preferences and watch history. Here is the next video we think you should check out!\n\nEpisode 1: Introduction to Collaborative Learning - https://www.cikguhub.my/courses/cikguhub/introduction_to_collaborative_learning/?cl=1"
body = """
<html>
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
  <title>CikguHub Recommendations</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
</head>

<body style="margin: 0; padding: 0;">
  <table role="presentation" border="0" cellpadding="0" cellspacing="0" width="100%">
    <tr>
      <td style="padding: 20px 0 30px 0;">

        <table align="center" border="0" cellpadding="0" cellspacing="0" width="600" style="border-collapse: collapse; border: 1px solid #cccccc;">
          <tr>
            <td align="center" bgcolor="#f3d239" style="padding: 40px 0 30px 0;">
              <img src="./cikgu_header.png" alt="learn, connect, grow" width="300" style="display: block;" />
            </td>
          </tr>
          <tr>
            <td bgcolor="#ffffff" style="padding: 40px 30px 40px 30px;">
              <table border="0" cellpadding="0" cellspacing="0" width="100%" style="border-collapse: collapse;">
                <tr>
                  <td style="color: #153643; font-family: Arial, sans-serif;">
                    <h1 style="font-size: 24px; margin: 0;">CikguHub's Recommended Videos!</h1>
                  </td>
                </tr>
                <tr>
                  <td style="color: #153643; font-family: Arial, sans-serif; font-size: 16px; line-height: 24px; padding: 20px 0 30px 0;">
                    <p style="margin: 0;">Thank you for being one of our first supporters! Part of being a CikguHub member is getting personalized video recommendations based on your preferences and watch history. Here is the next couple of videos we think you should check out:</p>
                  </td>
                </tr>
                <tr>
                  <td>
                    <table role="presentation" style="width:100%;border-collapse:collapse;border:0;border-spacing:0;">
                      <tr>
                        <td style="width:260px;padding:0;vertical-align:top;color:#153643;">
                          <p style="margin:0 0 25px 0;font-size:16px;line-height:24px;font-family:Arial,sans-serif;"><img src="./thumbnail_1.jpg" alt="Thumbnail 1" width="260" style="height:auto;display:block;" /></p>
                          <p style="margin:0;font-size:16px;line-height:24px;font-family:Arial,sans-serif;"><a href="https://www.cikguhub.my/courses/cikguhub/1676862439274/episode_2/?cl=1" style="color:#000000;text-decoration:underline;">5 components of Social-Emotional Learning</a></p>
                        </td>
                        <td style="width:20px;padding:0;font-size:0;line-height:0;">&nbsp;</td>
                        <td style="width:260px;padding:0;vertical-align:top;color:#153643;">
                          <p style="margin:0 0 25px 0;font-size:16px;line-height:24px;font-family:Arial,sans-serif;"><img src="./thumbnail_2.jpg" alt="Thumbnail 2" width="260" style="height:auto;display:block;" /></p>
                          <p style="margin:0;font-size:16px;line-height:24px;font-family:Arial,sans-serif;"><a href="https://www.cikguhub.my/courses/cikguhub/introduction_to_collaborative_learning/" style="color:#000000;text-decoration:underline;">Introduction to Collaborative Learning</a></p>
                        </td>
                      </tr>
                    </table>
                  </td>
                </tr>
              </table>
            </td>
          </tr>
          <tr>
            <td bgcolor="#f3d239" style="padding: 30px 30px;">
                <table border="0" cellpadding="0" cellspacing="0" width="100%" style="border-collapse: collapse;">
                <tr>
                  <td style="color: #00000; font-family: Arial, sans-serif; font-size: 14px;">
                    <p style="margin: 0;">&reg; CikguHub 2023<br/></p>
                  </td>
                  <td align="right">
                    <table border="0" cellpadding="0" cellspacing="0" style="border-collapse: collapse;">
                    </table>
                  </td>
                </tr>
              </table>
            </td>
          </tr>
        </table>

      </td>
    </tr>
  </table>
</body>
</html>
"""
sender = "cikguhub@gmail.com"
recipients = ["pherkok@gmail.com", "janice@myedvolution.com", "melissa@myedvolution.com", "jqs@umich.edu", "chriskok@umich.edu"]
# recipients = ["pherkok@gmail.com"]
password = keys['google_app_pass']

msg = MIMEMultipart()
msg['Subject'] = subject
msg['From'] = sender
msg['To'] = ', '.join(recipients)

# Sign off with Logo
attachment = 'cikgu_logo.png'

with open(attachment, 'rb') as fp:
    img = MIMEImage(fp.read())
    img.add_header('Content-Disposition', 'attachment', filename='cikgu_header.png')
    img.add_header('X-Attachment-Id', '0')
    img.add_header('Content-ID', '<0>')
    fp.close()
    msg.attach(img)

with open('is5MVZeiju8-HD.jpg', 'rb') as fp:
    img = MIMEImage(fp.read())
    img.add_header('Content-Disposition', 'attachment', filename='thumbnail_1.jpg')
    img.add_header('X-Attachment-Id', '1')
    img.add_header('Content-ID', '<1>')
    fp.close()
    msg.attach(img)

with open('ywyB-Iec0C4-HD.jpg', 'rb') as fp:
    img = MIMEImage(fp.read())
    img.add_header('Content-Disposition', 'attachment', filename='thumbnail_2.jpg')
    img.add_header('X-Attachment-Id', '21')
    img.add_header('Content-ID', '<2>')
    fp.close()
    msg.attach(img)

# Attach the HTML email
string = body

# Replace the relative path to images with ContentID
html_string = string.replace("./thumbnail_1.jpg", "cid:1")
html_string = html_string.replace("./thumbnail_2.jpg", "cid:2")
html_string = html_string.replace("./cikgu_header.png", "cid:0")

msg.attach(MIMEText(html_string, 'html', 'utf-8'))

msgText = MIMEText('<br/><br/>Best regards,<br/><b>%s</b><br/>' % ("The CikguHub Team"), 'html')   
msg.attach(msgText)   

smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
smtp_server.login(sender, password)
smtp_server.sendmail(sender, recipients, msg.as_string())
smtp_server.quit()

(221,
 b'2.0.0 closing connection ei10-20020a05690c2a0a00b00545afe68ecdsm3843490ywb.146 - gsmtp')